# Finetune DistilBERT for binary classification on the SMS Spam Collection
__________________________

***# This a lot to not say much ^^ We need a real life example specific to DistilBERT doing binary classification on the SMS Spam collection. I also need a clear explanation of why we're using BastionLab because by the end of this tutorial, I really have no idea of the benefit ^^***

This can be useful, for instance, when one wants to leverage large pre-trained models on a smaller private dataset, for instance, medical or financial records, and ensure data privacy regarding users' data.

BastionLabTorch is intended for scenarios where we have a data owner, for instance, a hospital, wanting to have third parties train models on their data, e.g. a startup, potentially on untrusted infrastructures, such as in the Cloud.

The strength of BastionLabTorch is that the data owner can have a high level of protection on data shared to a remote enclave hosted in the Cloud, and operated by the startup, thanks to memory isolation and encryption, and remote attestation from the use of secure enclaves. 

***# end comment***

In this notebook, we will illustrate how BastionLab works. We will use the publicly available dataset [SMS Spam Collection](https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset) to finetune a DistilBERT model on a classification task, to predict whether an email is spam or not.

In this guide, we will cover two phases:
- The offline phase, in which the data owner prepares the dataset and the data scientist prepares the model.
- The online phase, in which dataset and model are uploaded to the secure enclave. In the enclave, the uploaded model will be trained on the dataset. The data scientist can pull the weights once the training is over.

We largely followed [this tutorial](https://towardsdatascience.com/fine-tuning-bert-for-text-classification-54e7df642894) to prepare the data and pre-train the model we'll use in this example.

## Pre-requisites
__________________________

We need to have installed: 
- [BastionLab](https://bastionlab.readthedocs.io/en/latest/docs/getting-started/installation/)
- Hugging Face's [Transformers library](https://huggingface.co/docs/transformers/installation)
- [Pandas](LINK) ***# PANDAS NEEDS TO BE REPLACED BY POLARS***
- [IPython kernel](https://ipython.readthedocs.io/en/stable/install/kernel_install.html) for Jupyter
- [Jupyter Widgets](https://ipywidgets.readthedocs.io/en/7.x/user_install.html) to enable notebooks extensions ***# Verify if this is necessary***

In [ ]:
!pip install bastionlab
!pip install transformers pandas ipykernel ipywidgets

## Offline phase - Model and dataset preparation
__________________________________

In this section, data owner and data scientist will prepare their data and model so that they are ready-to-use for the training in BastionLab.

### Data owner's side: preparing the dataset

In this example, our data owner wants a third party data scientist to train an AI model to detect spam from emails.

Of course, in a real-world scenario, the data owner already posesses the data, but here, we will need to download one! We'll get the SPAM collection dataset and unzip it by running the following code block:

In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
!unzip smsspamcollection.zip

--2022-12-12 17:02:54--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203415 (199K) [application/x-httpd-php]
Saving to: ‘smsspamcollection.zip.2’

smsspamcollection.z 100%[===================>] 198.65K   251KB/s    in 0.8s    

2022-12-12 17:02:56 (251 KB/s) - ‘smsspamcollection.zip.2’ saved [203415/203415]

Archive:  smsspamcollection.zip
replace SMSSpamCollection? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


To upload the dataset to the BastionLab server, the data owner will need to prepare their dataset and make it available in a PyTorch `DataSet` object.

***# here we need a line explaining what is happening in the next block of code if what I wrote is wrong. Also, we need some comments in the code running the user through what is going on.***

***WE NEED TO CHANGE ALL PANDA TO POLARS***

In [2]:
import pandas as pd

file_path = "./SMSSpamCollection"

labels = []
texts = []
with open(file_path) as f:
    for line in f.readlines():
        split = line.split("\t")
        labels.append(1 if split[0] == "spam" else 0)
        texts.append(split[1])
df = pd.DataFrame({"label": labels, "text": texts})
print(len(df))
df.head()

5574


,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...\n
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


The data owner also needs to preprocess the data. We'll use a `DistilBertTokenizer` to obtain tensors ready to be fed to the model:

***# ADD COMMENTS IN CODE***

In [3]:
from transformers import DistilBertTokenizer
import torch

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

token_id = []
attention_masks = []
for sample in df.text.values:
    encoding_dict = tokenizer.encode_plus(
        sample,
        add_special_tokens=True,
        max_length=32,
        truncation=True,
        padding="max_length",
        return_attention_mask=True,
        return_tensors="pt",
    )
    token_id.append(encoding_dict["input_ids"])
    attention_masks.append(encoding_dict["attention_mask"])

token_id = torch.cat(token_id, dim=0).to(dtype=torch.int64)
attention_masks = torch.cat(attention_masks, dim=0).to(dtype=torch.int64)
labels = torch.tensor(df.label.values, dtype=torch.int64)

To make the training process faster in this demonstration, we'll only take a subset of the dataset, but you can choose to take the whole dataset if you want.

***# COMMENTS IN THE CODE***

In [4]:
import numpy as np

test_ratio = 0.2
limit = 64
nb_samples = len(token_id)

idx = np.arange(nb_samples)
np.random.shuffle(idx)

train_idx = idx[int(test_ratio * nb_samples) :][:limit]
test_idx = idx[: int(test_ratio * nb_samples)][:limit]

Finally, we create our training and validation `TensorDataset` objects. We'll use them to wrap our `Tensor` objects into a PyTorch `DataSet`.

***# LITTLE COMMENTS IN THE CODE***

In [5]:
from bastionlab.torch.utils import TensorDataset

train_set = TensorDataset(
    [token_id[train_idx], attention_masks[train_idx]], labels[train_idx]
)

validation_set = TensorDataset(
    [token_id[test_idx], attention_masks[test_idx]], labels[test_idx]
)

### Data scientist's side: preparing the model

On his side, the data scientist must prepare the DistilBERT language model. 

One important thing to know about BastionLab is that it supports models with an arbitrary number of inputs, but it only supports models with a single output. This is the first step we need to address as Hugging Face's models typically have several outputs (*logits, loss, etc*).

We'll use BastionLab's utility wrapper to select only one output of the model. In our case: the one that corresponds with the logits. 

***# COMMENTS IN THE CODE =)***

In [6]:
import torch.nn as nn


class MultipleOutputWrapper(nn.Module):
    """Utility wrapper to select one output of a model with multiple outputs.

    Args:
        module: A model with more than one output.
        output: Index of the output to retain.
    """

    def __init__(self, module: nn.Module, output: int = 0) -> None:
        super().__init__()
        self.inner = module
        self.output = output

    def forward(self, *args, **kwargs) -> torch.Tensor:
        output = self.inner.forward(*args, **kwargs)
        return output[self.output]

In [7]:
from transformers import DistilBertForSequenceClassification

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    output_attentions=False,
    output_hidden_states=False,
    torchscript=True,
)
model = MultipleOutputWrapper(
    model, 0
)  # This can be loaded from bastionlab.torch.utils

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

## Online phase - dataset and model upload and training
-------------------------------------------------------

Now that that both dataset and model are prepared, we can upload them securely to the secure enclave for the training.

### Data owner's side: uploading the dataset

We will connect to the BastionLab Torch instance using our `Connection` library. 

Once connected, we'll the `RemoteDataset()` method to upload the datasets inside the enclave. The method needs us to provide a name, and set a Differential Privacy budget. Here we put `1 000 000` an arbitrary number, but as a rule of thumb, it should be much lower, such as 4 or 8. 

***# Why don't we use the rule of thumb budget? and why should it be lower?***

> *To learn more about Differential Privacy and why it's important you use it, you [can read this article](**HAVE A LINK TO AN INTERESTING ARTICLE ON THE TOPIC WHILE WE MAKE OURS ^^**).*

In [9]:
from bastionlab import Connection

# The Data owner privately uploads their model online
client = Connection("localhost").client.torch

remote_dataset = client.RemoteDataset(
    train_set, validation_set, name="SMSSpamCollection", privacy_limit=1_000_000.0
)

Sending SMSSpamCollection: 100%|████████████████████| 35.6k/35.6k [00:00<00:00, 8.86MB/s]
Sending SMSSpamCollection (test): 100%|████████████████████| 35.6k/35.6k [00:00<00:00, 20.8MB/s]


### Data scientist's side: uploading the model and trigger training

It's finally time to train the model! 

The data scientist will use the `list_remote_datasets` endpoint to get a list of the available datasets on the server that they'll be able to use for training.

***# comments in the code! what is str(ds) blabla***

In [10]:
client = Connection("localhost").client.torch

remote_datasets = client.list_remote_datasets()

[str(ds) for ds in remote_datasets]

['SMSSpamCollection (5cd871636638c4cde41c672d735e2ab0af1628edc43ccd4bcdbc126c35e95fa1): size=64, desc=N/A']

The dataset uploaded previously is available as a `RemoteDataset ` object. It is a pointer to the remote dataset uploaded previously, **that contains only metadata and nothing else**. This allows the data scientist to play with remote datasets without users' data being exposed in the process.

In [11]:
remote_datasets[0]

To send the model to the server and set all the necessary training parameters, we'll use the `RemoteLearner()` method. 

To start training, we'll 'call the `fit` method on the `RemoteLearner` object with an appropriate number of epochs and Differential Privacy budget. 

Then we'll test the model directly on the server with the `test()` method.

***# what is appropriate? Are we sure epoch is clear to the reader?***

> Note that behind the scenes, a DP-SGD training loop will be used. 
>
>***# LINK TO PAPER? Why is that important and why are we mentionning this here?***

Finally, and it's the last step of this tutorial, we'll retrieve a local copy of the trained model once the training is complete. To do so, we'll use the `get_model()` method.

***# COMMENTS IN THE CODE. What are the given variables/arguments to the RemoteLearner method? what are the various arguments etc***

In [12]:
from bastionlab.torch.optimizer_config import Adam

# The Data Scientist discovers available datasets and uses one of them to train their model
client = Connection("localhost").client.torch
remote_learner = client.RemoteLearner(
    model,
    remote_datasets[0],
    max_batch_size=2,
    loss="cross_entropy",
    optimizer=Adam(lr=5e-5),
    model_name="DistilBERT",
)

remote_learner.fit(nb_epochs=2, eps=6.0)
remote_learner.test(metric="accuracy")

trained_model = remote_learner.get_model()

Sending DistilBERT: 100%|████████████████████| 268M/268M [00:05<00:00, 47.3MB/s] 
Epoch 1/2 - train:  12%|██▌                 | 4/32 [00:18<02:07,  4.54s/batch, cross_entropy=10.0000 (+/- 3656.3914)]  